## parse_elections 

Adapted from DexGroves's code.

Find the party-to-party transfer matrix for Northern Ireland's 2016 MLA election, using data from [Elections NI](http://electionsni.org.s3-website-eu-west-1.amazonaws.com/data/).

Table is read whereby row, column indicates transfers _from_ row and _to_ column. 

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os.path
import numpy as np 
import pandas as pd 
from glob import glob 
import csv

pd.options.display.max_columns = 100

%matplotlib inline

Make the _total_ matrix of party-to-party transfers countrywide. This is saved both as absolute vote totals (`transfers.csv`) and as a row-normalised relative amount (`transfers_rel.csv`).

In [75]:
def getConstitDestinations(file, whichSituation='all', fileToSaveEachRow=None):

    df = pd.DataFrame.from_csv(file)
    
    constituency = file.split('/')[-2]
    stillIn = df['Candidate_Id']
    stillIn = set(stillIn)
    voteDestinations = {}
    transferringCounts = {}
    for c in range(2,df.Count_Number.max()+1):
        #print(c)
        
        #Changed from ==c-1 to <c.
        transferring = df[(df.Count_Number==c) & (df.Occurred_On_Count<c) & (df.Transfers < 0)].Party_Name
        
        for cid in df[(df.Count_Number==c) & ((df.Status=='Elected') | (df.Status=='Excluded')) & (df.Occurred_On_Count<c)].Candidate_Id:
            if cid in stillIn:
                stillIn.remove(cid)
                #print 'removing',cid

        if transferring.size == 1:
            transferring = transferring.iloc[0]
            
#             if transferring in transferringCounts.keys():
#                 transferringCounts[transferring] += 1
#                 #TODO handle this case of two transfers from same party.
#                 #for now skip the second case
#                 print 'skipping a second/third',transferring
#                 continue
#             else:
#                 transferringCounts[transferring] = 1
            if transferring not in transferringCounts.keys():
                transferringCounts[transferring] = {}
            
            #print transferring,c

            totalVotesAvailable = float(-1*df[(df.Count_Number==c) & (df.Occurred_On_Count<c) & (df.Transfers < 0)].Transfers)
            #print(totalVotesAvailable)
            #print(dir(totalVotesAvailable))

            transfersReceived = df[(df.Count_Number==c)][['Candidate_Id','Party_Name','Transfers','Total_Votes']]
            transfersReceived = transfersReceived[transfersReceived.Transfers >= 0]  #exclude the transferrer
            transfersReceived = transfersReceived[transfersReceived.Candidate_Id.isin(stillIn)]  #exclude those not available
            transfersReceived = transfersReceived.assign(fracReceived = lambda x: x.Transfers/totalVotesAvailable)

            self_available = transferring in transfersReceived.Party_Name.tolist()
            if whichSituation == 'self' and not self_available:
                continue
            if whichSituation == 'noself' and self_available:
                continue
            
            #print('Transferring party = %s' % transferring)
            #print(transfersReceived[['Party_Name','fracReceived']])
            
            fracLost = 1 - transfersReceived.fracReceived.sum()

            newFractions = transfersReceived.apply(lambda x: x.to_dict(),axis=1).tolist()
            #print newFractions
            for item in newFractions:
                party = item['Party_Name']
                if transferring in voteDestinations.keys():
                    if party in voteDestinations[transferring].keys():
                        voteDestinations[transferring][party] += item['fracReceived']
                    else:
                        voteDestinations[transferring][party] = item['fracReceived']
                else:
                    voteDestinations[transferring] = {party:item['fracReceived']}
                
                #Output individual transfer records
                item['Constituency'] = constituency
                item['Self_Available'] = self_available
                item['Count_Number'] = c
                item['Transferrer'] = transferring
                item['Target'] = party
                item['Transferred_Votes'] = totalVotesAvailable
                item['Target_Total_Votes_So_Far'] = item['Total_Votes']
                item['Number_Of_Targets_Left'] = len(stillIn)
                if fileToSaveEachRow is not None:
                    with open(fileToSaveEachRow,'a') as csvfile:
                        mywriter = csv.writer(csvfile)
                        mywriter.writerow([item[k] for k in header])

            if 'votes_lost' in voteDestinations[transferring].keys():
                voteDestinations[transferring]['votes_lost'] += fracLost
            else:
                voteDestinations[transferring]['votes_lost'] = fracLost
            
            #keep track of how many times the donor party occurred
            for party in set([item['Target'] for item in newFractions]+['votes_lost']):
                if party in transferringCounts[transferring].keys():
                    transferringCounts[transferring][party] += 1
                else:
                    transferringCounts[transferring][party] = 1
            
            #if transferring == 'Ulster Unionist Party' and whichSituation=='self':
            #    print voteDestinations[transferring]['Ulster Unionist Party']
            
            #print voteDestinations[transferring]
            if sum(voteDestinations[transferring].values()) < 0.9999:
                print('missing some',sum(voteDestinations[transferring].values()))
        else:
            #print(transferring.size)
            pass
            #TODO
    
    for donor in voteDestinations.keys():
        for recip in voteDestinations[donor].keys():
            #print voteDestinations[donor][recip],transferringCounts[donor][recip]
            voteDestinations[donor][recip] /= transferringCounts[donor][recip]
            
    return voteDestinations
           
#TODO: could also note count number applying to each transferring. Vote more likely to be lost
#  if it is being passed from the party at a high count number


In [3]:
mine = getConstitDestinations('data/2016_archive_datapackage/constituency/strangford/Count.csv', whichSituation='noself')
for key in mine:
    print(sum(mine[key].values()))
    print(key,mine[key])
#for filename in glob('data/2016_archive_datapackage/constituency/*/Count.csv'):
#    voteDestinations = getConstitDestinations(filename, whichSituation='self')
    #print voteDestinations


1.0
Sinn Fein {'Alliance Party': 0.1404833836858006, 'Democratic Unionist Party': 0.013595166163141995, 'Social Democratic and Labour Party': 0.6253776435045317, 'Traditional Unionist Voice': 0.0015105740181268882, 'UK Independence Party': 0.01661631419939577, 'Green Party': 0.07099697885196375, 'Independent': 0.010574018126888218, 'Ulster Unionist Party': 0.0, 'votes_lost': 0.12084592145015105}
1.0
UK Independence Party {'Alliance Party': 0.07930607187112763, 'Democratic Unionist Party': 0.29491945477075593, 'Social Democratic and Labour Party': 0.02973977695167286, 'Traditional Unionist Voice': 0.21189591078066913, 'Green Party': 0.061957868649318466, 'Independent': 0.08302354399008674, 'Ulster Unionist Party': 0.12391573729863693, 'votes_lost': 0.11524163568773238}
1.0
Green Party {'Alliance Party': 0.5685522531160115, 'Democratic Unionist Party': 0.05560882070949186, 'Social Democratic and Labour Party': 0.09587727708533078, 'Traditional Unionist Voice': 0.015340364333652923, 'Inde

In [76]:
#TODO: normalisation OK? UUP self prob seems low at 0.63

#Separate matrices for when self transfer available and not

def makeOverallMatrix(year, whichSituation='all', fileToSaveEachRow=None):
    dests = ['Alliance Party','Animal Welfare Party','Cannabis Is Safer Than Alcohol',
         'Cross-Community Labour Alternative','Democracy First','Democratic Unionist Party','Green Party',
         'Independent','Labour Alternative','NI Conservatives','NI Labour Representation Committee',
         'Northern Ireland First','People Before Profit Alliance','Progressive Unionist Party','Sinn Fein',
         'Social Democratic and Labour Party','South Belfast Unionists','Traditional Unionist Voice',
         'UK Independence Party','Ulster Unionist Party','Workers Party','votes_lost']

    overallDests = {}

    if whichSituation=='self':
        outfile = 'transferProbs%i/transferMatrix_whenSelfAvailable_nationwide.csv' % year
        outfile2 = 'transferProbs%i/transferMatrix_whenSelfAvailable_%s.csv'
    elif whichSituation=='noself':
        outfile = 'transferProbs%i/transferMatrix_whenSelfNotAvailable_nationwide.csv' % year
        outfile2 = 'transferProbs%i/transferMatrix_whenSelfNotAvailable_%s.csv'
    else:
        outfile = 'transferProbs%i/transferMatrix_all_nationwide.csv' % year
        outfile2 = 'transferProbs%i/transferMatrix_all_%s.csv'

    for filename in glob('data/%i_archive_datapackage/constituency/*/Count.csv' % year):
        
        voteDestinations = getConstitDestinations(filename, whichSituation=whichSituation, 
                                                  fileToSaveEachRow=fileToSaveEachRow)
        
        constit = filename.split('/')[3]
        
        with open(outfile2 % (year,constit),'w') as csvfile:
            mywriter = csv.writer(csvfile)
            mywriter.writerow(dests)
            for donor in dests:
                newlist = [donor]
                if donor in voteDestinations.keys():
                    for dest in dests:
                        if dest in voteDestinations[donor].keys():
                            #print donor,dest,overallDests[donor][dest][1] / overallDests[donor][dest][0]
                            newlist.append('%.5f' % voteDestinations[donor][dest])
                        else:
                            #print donor,dest,'NaN'
                            newlist.append('NaN')
                else:
                    newlist += ['NaN']*len(dests)
                mywriter.writerow(newlist)

        for donor in voteDestinations.keys():
            if donor in overallDests.keys():
                for dest in voteDestinations[donor]:
                    if dest in overallDests[donor].keys():
                        overallDests[donor][dest][0] += 1
                        overallDests[donor][dest][1] += voteDestinations[donor][dest]
                    else:
                        overallDests[donor][dest] = [1,voteDestinations[donor][dest]]
            else:
                overallDests[donor] = {}
                for dest in voteDestinations[donor].keys():
                    overallDests[donor][dest] = [1,voteDestinations[donor][dest]]

    with open(outfile,'w') as csvfile:
        mywriter = csv.writer(csvfile)
        mywriter.writerow(dests)
        for donor in overallDests.keys():
            newlist = [donor]
            normFactor = 1 / sum([overallDests[donor][dest][1]/overallDests[donor][dest][0] for dest in overallDests[donor]])
            #print donor,normFactor
            #print overallDests[donor]
            for dest in dests:
                if dest in overallDests[donor].keys():
                    newlist.append('%.5f' % (normFactor * overallDests[donor][dest][1] / overallDests[donor][dest][0]))
                else:
                    newlist.append('NaN')
            mywriter.writerow(newlist)
        
makeOverallMatrix(2016,whichSituation='self')
makeOverallMatrix(2016,whichSituation='noself')

header = ['Transferrer','Target','Constituency','Count_Number','Self_Available','Number_Of_Targets_Left',
          'Transferred_Votes','Target_Total_Votes_So_Far','fracReceived']
with open('transfersByRow_2016_all.csv','w') as f:
    f.write(','.join(header)+'\n')
makeOverallMatrix(2016,whichSituation='all',fileToSaveEachRow='transfersByRow_2016_all.csv')

140

In [45]:
getConstitDestinations('data/2016_archive_datapackage/constituency/belfast-east/Count.csv',whichSituation='all',fileToSaveEachRow='this.txt')

Transferring party = Democratic Unionist Party
Transferring party = Alliance Party
Transferring party = NI Conservatives
Transferring party = Labour Alternative
Transferring party = UK Independence Party
Transferring party = Sinn Fein
Transferring party = Traditional Unionist Voice
Transferring party = Independent
Transferring party = Ulster Unionist Party
Transferring party = Progressive Unionist Party
Transferring party = Alliance Party


{'Alliance Party': {'Alliance Party': 0.7985667854657619,
  'Democratic Unionist Party': 0.02630973773306521,
  'Green Party': 0.06871507140920852,
  'Independent': 0.00456140350877193,
  'Labour Alternative': 0.013859649122807018,
  'NI Conservatives': 0.006666666666666666,
  'Progressive Unionist Party': 0.015263157894736841,
  'Sinn Fein': 0.01,
  'Traditional Unionist Voice': 0.0015789473684210528,
  'UK Independence Party': 0.0036842105263157894,
  'Ulster Unionist Party': 0.03929824561403509,
  'votes_lost': 0.05895226504108725},
 'Democratic Unionist Party': {'Alliance Party': 0.005638766519823788,
  'Democratic Unionist Party': 0.8519823788546257,
  'Green Party': 0.006167400881057268,
  'Independent': 0.006519823788546255,
  'Labour Alternative': 0.000881057268722467,
  'NI Conservatives': 0.0015859030837004405,
  'Progressive Unionist Party': 0.01762114537444934,
  'Sinn Fein': 0.0,
  'Social Democratic and Labour Party': 0.0033480176211453743,
  'Traditional Unionist Voice':